In [24]:
import requests
import asyncio
import logging

import tornado.gen

import waterbutler.core.streams
import waterbutler.core.exceptions
from waterbutler.core.utils import make_provider
from waterbutler.core.streams import ResponseStreamReader
import aiohttp

In [25]:
# specific to the provider
auth = {
	'name':'himanshu',
	'email':'ho2es@virginia.edu'
}
# specific to provider
credentials = {
	'token': 'eyJhbGciOiJIUzUxMiJ9.ZXlKaGJHY2lPaUprYVhJaUxDSmxibU1pT2lKQk1USTRRMEpETFVoVE1qVTJJbjAuLlZERkg3blRmN1JwS3otWHl5V2NyYncuMFhiOFZuRzFMODVyMzE4WERzc2VPWFI5X3lLOUtlR1otcEYtdXRSRWwzWTl6bUFNU0hSTmhXQ1pNazRjOWpxYXpsU2w1QWp3THBqdzA2cjAxaFBlOHZHdUdnZHpmLTgzUW80WUVhQjJ5d2o4U2RQLVljZUlLS3VraEZjUk5LZWNrai1Td3YzdmxmTTYzSTBITEQxUnNyYzB0bm1qTGpoV0V0NDZrT1pNZFN3LnZfOU9CV285ZDJLU21fOWd2ZWs1bWc._mPnS2zb6Cm-f-uSjzI3OVvUz7IfsxL3IPB_IShRHwQNyNw-zZM1_RrMMHSzmq9s0VS8bAdAg4JfDmMB438uIg'
}
# specific to provider
settings = {
	'folder': {
		'id': 'someid',
		'name': '/myfolder/'
	}
}




In [26]:
# provider to write/read from the local file system
local_filesystem_provider = make_provider(
	name='filesystem',
    auth={},
	credentials={},
	settings={'folder': '/tmp/blah'}
)


In [32]:
def upload(file_like):
    pass

@asyncio.coroutine
def download(file_name, token):
    """
    This method uploads a file to the local_file_system_provider from the waterbutler file response.
    Using a waterbutler provider allows it to use the response stream reader AND allows us to do this asynchronously.
    """
    
    path = yield from local_filesystem_provider.validate_path(file_name)
    
    response = yield from aiohttp.request(
        'GET',
        'http://localhost:7777/file?path=/blah.txt&nid=nq782&provider=osfstorage',
        headers={'Authorization': 'Bearer {}'.format(token)}
    )
    yield from local_filesystem_provider.upload(
        ResponseStreamReader(response, unsizable=True),
        path,
    )

loop = asyncio.get_event_loop()
loop.run_until_complete(download('/blah.txt', credentials['token']))

/blah.txt


In [ ]:
@asyncio.coroutine
def create_local_file_folder(local, remote): 
    local_filesystem_provider = make_provider(
        name='filesystem',
        auth={},
        credentials={},
        settings={'folder': local.path}
    )
    #validates and returns a WaterButlerPath object
    path = yield from local_filesystem_provider.validate_path(remote['name'])
    #wait until response is attained from given url.
    response = yield from make_request(url=remote['links']['self'])
    if response.ok:
        #todo: update local model accordingly.
        
        # stream the file response to where it belongs.
        yield from local_file_system_provider.upload(
            ResponseStreamReader(response, unsizable=True),
            path,
        )

@asyncio.coroutine
def make_request(url):
    yield aiohttp.request(
        'GET',
        url,
        headers={'Authorization': 'Bearer {}'.format(self.token)}
    )

In [ ]:
@asyncio.coroutine
def create_local_file_folder(self, remote_file_folder, local_parent_folder, local_node):
        print('creating local file folder')
        assert remote_file_folder is not None
        assert remote_file_folder['type'] == 'files'
        assert isinstance(local_parent_folder, File) or local_parent_folder is None
        assert local_parent_folder is None or (local_parent_folder.type == File.FOLDER)
        assert isinstance(local_node, Node)


        #create local file folder in db
        type = File.FILE if remote_file_folder['item_type']=='file' else File.FOLDER
        new_file_folder = File(
            name=remote_file_folder['name'],
            type=type,
            osf_id=self.get_id(remote_file_folder),
            provider=remote_file_folder['provider'],
            osf_path=remote_file_folder['path'],
            user=self.user,
            parent=local_parent_folder,
            node=local_node)
        self.save(new_file_folder)

        #create local file/folder on actual system
        local_filesystem_provider = make_provider(
            name='filesystem',
            auth={},
            credentials={},
            settings={'folder': local.path}
        )
        #validates and returns a WaterButlerPath object
        path = yield from local_filesystem_provider.validate_path(remote['name'])
        #wait until response is attained from given url.
        response = yield from make_request(url=remote['links']['self'])
        if response.ok:
            #todo: update local model accordingly.

            # stream the file response to where it belongs.
            yield from local_file_system_provider.upload(
                ResponseStreamReader(response, unsizable=True),
                path,
            )
        
        if not os.path.exists(new_file_folder.path):
            if type == File.FILE:
                resp = requests.get(remote_file_folder['links']['self'], headers=self.headers)
                with open(new_file_folder.path, 'wb') as fd:
                    for chunk in resp.iter_content(2048): #todo: which is better? 1024 or 2048? Apparently, not much difference.
                        fd.write(chunk)
            elif type == File.FOLDER:
                os.makedirs(new_file_folder.path)
            else:
                raise ValueError('file type is unknown')
        return new_file_folder